In [1]:
import numpy as np
import pandas as pd

In [2]:
analys_personal_data = pd.read_csv('../../../Data/For_analysis_and_modeling/2nd_quadrimester/analys_personal_data.csv',sep='|')

In [3]:
analys_personal_data.head()

,expediente,cod_plan,des_plan,anio_apertura_expediente,abandona,convocatoria_acceso,des_acceso,nota_admision_def,sexo,edad_acceso,distance,becario,cum_pass_ratio,cum_absent_ratio,std_deviation,cum_median,cum_pass_median,cum_fail_median
0,2,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Selectividad,6.836,D,25,300.6305,S,0.235294,0.176471,1.990064,4.175,5.7,3.84
1,3,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,Formación Profesional,6.836,H,31,667.4115,S,0.312500,0.562500,1.566486,5.000,5.0,2.50
2,4,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Formación Profesional,6.836,D,28,91.0020,N,0.166667,0.444444,1.345324,3.825,5.2,2.53
3,5,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,Selectividad,6.730,D,25,91.0020,N,0.263158,0.315789,1.850120,3.670,5.4,2.50
4,6,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Selectividad,6.836,H,25,91.0020,N,1.000000,0.000000,1.339003,7.000,7.0,0.00


In [4]:
def bin_variance_report(series, bin_edges):
    """Devuelve métricas intra/inter-bin para una serie numérica."""
    groups = pd.cut(series, bins=bin_edges, include_lowest=True)
    n_total    = series.size
    global_var = series.var(ddof=0)

    # Varianza interna ponderada
    intra = sum(g.var(ddof=0) * g.size for _, g in series.groupby(groups)) / n_total

    # Varianza entre bins
    means  = series.groupby(groups).mean()
    counts = series.groupby(groups).size()
    grand_mean = series.mean()
    inter = sum(counts[g] * (means[g] - grand_mean)**2 for g in means.index) / n_total

    return {
        "intra_variance": intra,
        "inter_variance": inter,
        "F_ratio": inter / intra if intra else np.nan,
        "R2_explained": inter / global_var if global_var else np.nan,
    }

# --- Límites que definiste --------------------------------------------------
edges = {
    "nota_admision_def": np.array([5, 6.5, 8, 9.5, 10, 11.5, 13, 14]),
    "distance":          np.array([0, 45, 90, 135, 250, 500, 1892]),
    "cum_median":        np.array([0, 1.5, 3, 4.5, 6, 7.5, 9, 10]),
    "cum_pass_median":   np.array([0, 4.5, 6, 7.5, 9, 10]),
    "cum_fail_median":   np.array([0, 1.5, 3, 5]),
    "cum_pass_ratio":    np.array([0, 0.25, 0.5, 0.75, 1]),
    "cum_absent_ratio":  np.array([0, 0.25, 0.5, 0.75, 1]),
    "std_deviation":     np.array([0, 0.05, 1.15, 1.25, 2.15, 3.25]),
    "edad_acceso":       np.array([18, 20, 25, 30, 35, 60])
}

# --- Generar los informes ---------------------------------------------------
reports = {
    col: bin_variance_report(analys_personal_data[col], edge_arr)
    for col, edge_arr in edges.items()
}

report_df = (
    pd.DataFrame(reports).T            # variables como filas
      .sort_values("R2_explained", ascending=False))

print(report_df)


                   intra_variance  inter_variance    F_ratio  R2_explained
cum_pass_median          0.113517        5.732911  50.502714      0.980584
cum_median               0.185502        3.500254  18.869052      0.949670
cum_absent_ratio         0.006069        0.073035  12.034328      0.923280
cum_pass_ratio           0.006206        0.060786   9.794204      0.907358
nota_admision_def        0.203781        1.806111   8.862983      0.898611
distance              2588.685412    19459.485860   7.517130      0.882590
std_deviation            0.053724        0.351748   6.547279      0.867502
edad_acceso              1.843093       11.325504   6.144835      0.860039
cum_fail_median          0.204864        1.018388   4.971057      0.832525
